# 03 - Dataset Assembley

This notebook consolidates individual stock-level feature sets into a single unified dataset. Each company's data, enriched with technical indicators and fundamental ratios, is loaded from preprocessed CSV files.

The output is a merged DataFrame containing all stocks, aligned by date and annotated with sector and industry metadata. This dataset is used as input for machine learning models in the next stage.


In [ ]:
# Import necessary libraries
import os
import pandas as pd

## 1. Load S&P 500 Company Metadata

Read the S&P 500 constituents list which contains the stock ticker, sector, and sub-industry for each company.


In [2]:
sp500 = pd.read_csv("S&P_500_Companies.csv")

In [3]:
sp500

,Symbol,Security,GICS Sector,GICS Sub-Industry
0,MMM,3M,Industrials,Industrial Conglomerates
1,AOS,A. O. Smith,Industrials,Building Products
2,ABT,Abbott,Health Care,Health Care Equipment
3,ABBV,AbbVie,Health Care,Biotechnology
4,ACN,Accenture,Information Technology,IT Consulting & Other Services
...,...,...,...,...
498,YUM,Yum! Brands,Consumer Discretionary,Restaurants
499,ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments
500,ZBH,Zimmer Biomet,Health Care,Health Care Equipment
501,ZION,Zions Bancorporation,Financials,Regional Banks


## 2. Load and Annotate Individual Stock Feature Sets

Loop through each stock in the S&P 500 list. Load its processed feature set CSV, annotate it with metadata, and collect all into a list.


In [4]:
def load_stock_feature_sets(sp500_df, base_path="dataset\\Stocks\\"):
    """
    Load stock feature sets for each symbol listed in the given S&P 500 dataframe.

    Parameters:
        sp500_df (pd.DataFrame): A DataFrame containing S&P 500 data, including 'Symbol' and
                                 'GICS Sector' columns.
        base_path (str): The path to the folder containing individual stock subfolders with CSV files.

    Returns:
        list of pd.DataFrame: A list of DataFrames loaded from the CSV files, 
                              each with added Symbol, Sector, and Sub-Industry columns.
    """
    dfs = []

    for index, row in sp500_df.iterrows():
        symbol = row['Symbol'].strip()
        sector = row['GICS Sector']
        csv_file_path = os.path.join(base_path, f"{symbol}\\{symbol}_full_feature_set.csv")
        
        if os.path.exists(csv_file_path):
            print(f"Loading {symbol} data from {csv_file_path}")
            try:
                df_csv = pd.read_csv(csv_file_path)
                df_csv['Symbol'] = symbol
                df_csv['Sector'] = sector
                dfs.append(df_csv)
                print(f"Loaded {symbol} data with shape: {df_csv.shape}")
            except Exception as e:
                print(f"Error reading file for {symbol}: {e}")

    return dfs

In [5]:
dfs = load_stock_feature_sets(sp500)

Loading AAPL data from dataset\Stocks\AAPL\AAPL_full_feature_set.csv
Loaded AAPL data with shape: (4638, 38)
Loading XOM data from dataset\Stocks\XOM\XOM_full_feature_set.csv
Loaded XOM data with shape: (4331, 38)
Loading JNJ data from dataset\Stocks\JNJ\JNJ_full_feature_set.csv
Loaded JNJ data with shape: (4331, 38)
Loading NVDA data from dataset\Stocks\NVDA\NVDA_full_feature_set.csv
Loaded NVDA data with shape: (4560, 38)
Loading PG data from dataset\Stocks\PG\PG_full_feature_set.csv
Loaded PG data with shape: (4706, 38)


## 3. Compile All Feature Sets

Concatenate all individual stock dataframes into a single dataset. Format and sort the combined DataFrame for downstream analysis.


In [6]:
def compile_stock_dataframes(dfs):
    """
    Compile and prepare a unified stock dataset from a list of DataFrames.
    
    Steps:
    - Concatenate all input DataFrames.
    - Move the last two columns (assumed to be 'Symbol' and 'Sector') to the front.
    - Convert 'Date' to datetime format.
    - Sort by 'Date' and 'Symbol'.
    - Set 'Date' as the index.

    Parameters:
        dfs (list of pd.DataFrame): List of individual stock DataFrames.

    Returns:
        pd.DataFrame: A cleaned, sorted, and indexed combined dataset.
    """
    if not dfs:
        raise ValueError("The input list of DataFrames is empty.")

    # Merge all stock DataFrames
    dataset = pd.concat(dfs, ignore_index=True)

    # Move last two columns to the front (e.g., Symbol and Sector)
    columns = dataset.columns.tolist()
    reordered_columns = columns[-2:] + columns[:-2]
    dataset = dataset[reordered_columns]

    # Ensure 'Date' exists and is in datetime format
    if 'Date' not in dataset.columns:
        raise KeyError("Column 'Date' is missing from the dataset.")
    dataset['Date'] = pd.to_datetime(dataset['Date'])

    # Sort by 'Date' and 'Symbol'
    dataset = dataset.sort_values(by=['Date', 'Symbol'])

    # Set 'Date' as index
    dataset.set_index('Date', inplace=True)

    return dataset


In [7]:
dataset = compile_stock_dataframes(dfs)
dataset

,Symbol,Sector,open_adj,high_adj,low_adj,close_adj,volume_adj,dividends,stock_splits,sma_10,...,close_raw,volume_raw,roe,pe_ratio,pb_ratio,eps_growth_qoq,eps_growth_yoy,ev_ebitda,de_ratio,fcf_yield
Date,,,,,,,,,,,,,,,,,,,,,
2006-09-01,PG,Consumer Staples,36.455025,36.513832,36.313885,36.407978,4132300,0.00,0.0,36.014553,...,61.91,4132300.0,0.030171,94.884738,2.904178,-0.061759,0.044869,66.807390,1.157039,0.047664
2006-09-05,PG,Consumer Staples,36.166855,36.278589,35.984550,36.202141,6288700,0.00,0.0,36.060420,...,61.56,6288700.0,0.030171,96.255465,2.904178,-0.061759,0.044869,67.513398,1.157039,0.047664
2006-09-06,PG,Consumer Staples,36.202144,36.266831,35.955149,36.131573,4595800,0.00,0.0,36.083355,...,61.44,4595800.0,0.030171,95.900091,2.921415,-0.061759,0.044869,67.330359,1.157039,0.047664
2006-09-07,PG,Consumer Staples,36.396214,36.478545,36.072771,36.184505,7174000,0.00,0.0,36.119229,...,61.53,7174000.0,0.030171,95.510872,2.948577,-0.061759,0.044869,67.129888,1.157039,0.047664
2006-09-08,PG,Consumer Staples,36.660858,36.660858,35.931638,35.955162,5725900,0.00,0.0,36.124523,...,61.14,5725900.0,0.030171,96.137007,2.951188,-0.061759,0.044869,67.452385,1.157039,0.047664
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-05-15,XOM,Energy,106.489998,108.650002,105.970001,108.580002,15004300,0.99,0.0,105.910549,...,108.58,15004291.0,0.028587,65.041185,1.725992,0.013535,-0.118237,37.999238,0.674937,0.066437
2025-05-16,JNJ,Health Care,149.669998,151.490005,149.220001,151.330002,6268118,0.00,0.0,152.653001,...,151.33,8017034.0,0.140816,33.768807,5.481480,2.210136,-0.574349,28.737817,1.479497,0.057003
2025-05-16,NVDA,Information Technology,136.250000,136.309998,133.460007,135.399994,218314761,0.00,0.0,123.693999,...,135.40,218893837.0,0.278480,105.473852,38.043827,0.147227,1.462352,91.154791,0.406848,0.020433


In [8]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 22566 entries, 2006-09-01 to 2025-05-16
Data columns (total 37 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Symbol          22566 non-null  object 
 1   Sector          22566 non-null  object 
 2   open_adj        22566 non-null  float64
 3   high_adj        22566 non-null  float64
 4   low_adj         22566 non-null  float64
 5   close_adj       22566 non-null  float64
 6   volume_adj      22566 non-null  int64  
 7   dividends       22566 non-null  float64
 8   stock_splits    22566 non-null  float64
 9   sma_10          22566 non-null  float64
 10  sma_50          22566 non-null  float64
 11  ema_12          22566 non-null  float64
 12  ema_26          22566 non-null  float64
 13  macd            22566 non-null  float64
 14  macd_signal     22566 non-null  float64
 15  rsi_14          22566 non-null  float64
 16  bb_upper        22566 non-null  float64
 17  bb_middle     

In [ ]:
dataset.to_csv("dataset\\dataset.csv", index=True)